In [3]:
import os
import requests
import gzip
import shutil
import rasterio
import geopandas as gpd
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
from datetime import datetime, timedelta
from tqdm import tqdm  # Progress bar for better visualization

# CHIRPS HTTPS base URL
BASE_URL = "https://data.chc.ucsb.edu/products/CHIRPS-2.0/africa_daily/tifs/p05/"

# Define date range
start_year = 2022  # Change to desired start year
end_year = 2024    # Change to desired end year

# Path to shapefile (update this)
shapefile_path = r"C:\Users\jmsch\OneDrive\Documenten\Studie\Civiele Techniek\Environmental Engineering\Year 2\Afstuderen\NBS Nakuru Kenia\2. QGIS + Wflow\RESULTS\Delineated_Basin_Nakuru.gpkg"

# Load shapefile
gdf = gpd.read_file(shapefile_path)
geom = [gdf.geometry.unary_union]  # Convert shapefile to geometry

# Create main download directory
download_dir = "CHIRPS_Downloads"
os.makedirs(download_dir, exist_ok=True)

# Loop through each year
for year in range(start_year, end_year + 1):
    year_dir = os.path.join(download_dir, str(year))
    os.makedirs(year_dir, exist_ok=True)

    start_date = datetime(year, 1, 1)
    end_date = datetime(year, 12, 31)

    # Loop through each day
    current_date = start_date
    while current_date <= end_date:
        file_name = f"chirps-v2.0.{current_date.strftime('%Y.%m.%d')}.tif.gz"
        file_url = f"{BASE_URL}{year}/{file_name}"
        local_gz_path = os.path.join(year_dir, file_name)
        local_tif_path = local_gz_path[:-3]  # Remove .gz extension

        # Download the file if not already downloaded
        if not os.path.exists(local_gz_path) and not os.path.exists(local_tif_path):
            print(f"Downloading {file_name}...")
            response = requests.get(file_url, stream=True)
            if response.status_code == 200:
                with open(local_gz_path, "wb") as file:
                    for chunk in response.iter_content(chunk_size=1024):
                        file.write(chunk)
            else:
                print(f"File not found: {file_url} (Skipping)")
                current_date += timedelta(days=1)
                continue  # Skip to next day

        # Extract .tif from .gz
        if not os.path.exists(local_tif_path):  # Avoid re-extracting
            print(f"Extracting {file_name}...")
            with gzip.open(local_gz_path, 'rb') as f_in, open(local_tif_path, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)

        # Optionally, delete the .gz file after extraction
        #os.remove(local_gz_path)

        # Reproject raster to EPSG:32737
        reprojected_tif_path = local_tif_path.replace(".tif", "_reprojected.tif")
        if not os.path.exists(reprojected_tif_path):  # Avoid reprocessing
            print(f"Reprojecting {file_name} to EPSG:32737...")
            with rasterio.open(local_tif_path) as src:
                # Calculate the transform and CRS for reprojecting
                transform, width, height = calculate_default_transform(
                    src.crs, 'EPSG:32737', src.width, src.height, *src.bounds)
                kwargs = src.meta.copy()
                kwargs.update({
                    'crs': 'EPSG:32737',
                    'transform': transform,
                    'width': width,
                    'height': height
                })

                # Reproject to the new CRS (EPSG:32737)
                with rasterio.open(reprojected_tif_path, 'w', **kwargs) as dst:
                    reproject(
                        source=rasterio.band(src, 1),
                        destination=rasterio.band(dst, 1),
                        src_transform=src.transform,
                        src_crs=src.crs,
                        dst_transform=transform,
                        dst_crs='EPSG:32737',
                        resampling=Resampling.nearest)

                print(f"Reprojected raster saved: {reprojected_tif_path}")

            # Optionally, remove the full-size file after reprojecting
            os.remove(local_tif_path)

        # Clip the reprojected raster to the shapefile extent
        clipped_tif_path = reprojected_tif_path.replace(".tif", "_clipped.tif")
        if not os.path.exists(clipped_tif_path):  # Avoid reprocessing
            print(f"Clipping {file_name}...")
            with rasterio.open(reprojected_tif_path) as src:
                out_image, out_transform = mask(src, geom, crop=True)
                out_meta = src.meta.copy()

                # Update metadata
                out_meta.update({
                    "driver": "GTiff",
                    "height": out_image.shape[1],
                    "width": out_image.shape[2],
                    "transform": out_transform
                })

                # Save clipped raster
                with rasterio.open(clipped_tif_path, "w", **out_meta) as dest:
                    dest.write(out_image)

                print(f"Saved clipped raster: {clipped_tif_path}")

            # Optionally, remove the reprojected file after clipping
            os.remove(reprojected_tif_path)

        current_date += timedelta(days=1)

print("Download, Extraction, Reprojection & Clipping Complete!")





C:\Users\jmsch\AppData\Local\Temp\ipykernel_29780\2947526739.py:24: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom = [gdf.geometry.unary_union]  # Convert shapefile to geometry


Clipping chirps-v2.0.2022.09.25.tif.gz...
Saved clipped raster: CHIRPS_Downloads\2022\chirps-v2.0.2022.09.25_reprojected_clipped.tif
Extracting chirps-v2.0.2022.09.26.tif.gz...
Reprojecting chirps-v2.0.2022.09.26.tif.gz to EPSG:32737...
Reprojected raster saved: CHIRPS_Downloads\2022\chirps-v2.0.2022.09.26_reprojected.tif
Clipping chirps-v2.0.2022.09.26.tif.gz...
Saved clipped raster: CHIRPS_Downloads\2022\chirps-v2.0.2022.09.26_reprojected_clipped.tif
Extracting chirps-v2.0.2022.09.27.tif.gz...
Reprojecting chirps-v2.0.2022.09.27.tif.gz to EPSG:32737...
Reprojected raster saved: CHIRPS_Downloads\2022\chirps-v2.0.2022.09.27_reprojected.tif
Clipping chirps-v2.0.2022.09.27.tif.gz...
Saved clipped raster: CHIRPS_Downloads\2022\chirps-v2.0.2022.09.27_reprojected_clipped.tif
Extracting chirps-v2.0.2022.09.28.tif.gz...
Reprojecting chirps-v2.0.2022.09.28.tif.gz to EPSG:32737...
Reprojected raster saved: CHIRPS_Downloads\2022\chirps-v2.0.2022.09.28_reprojected.tif
Clipping chirps-v2.0.2022.09